In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import py4DSTEM
import pyprismatic
from pyprismatic.process import Cc

## Simulation

In [ ]:
cc_folder = "outputs/cc/"
pathlib.Path(cc_folder).mkdir(parents=True, exist_ok=True) 

In [ ]:
fp_atoms = "structure_files/WS2_moire_7_3_degree.xyz"
fp_20  = cc_folder + "df_series_20kv.h5"
fp_100 = cc_folder + "df_series_100kv.h5"

In [ ]:
sim_meta = pyprismatic.Metadata()
sim_meta.filenameAtoms = fp_atoms
sim_meta.filenameOutput = fp_100

sim_meta.algorithm = "p"
sim_meta.interpolationFactorX = 2
sim_meta.interpolationFactorY = 2
sim_meta.E0 = 100
sim_meta.realspacePixelSize = 0.1
sim_meta.probeSemiangle = 21

sim_meta.probeStep = 0.2
sim_meta.scanWindowX = (0.0, 0.5)
sim_meta.scanWindowY = (0.0, 0.5)
sim_meta.probeDefocus_min = -30.0
sim_meta.probeDefocus_max = 30.0
sim_meta.probeDefocus_step = 0.9375

sim_meta.potential3D = False
sim_meta.save3DOutput = True
sim_meta.includeThermalEffects = False

In [ ]:
if not pathlib.Path(sim_meta.filenameOutput).exists():
    sim_meta.go()

In [ ]:
sim_meta.filenameOutput = fp_20
sim_meta.E0 = 20
sim_meta.probeDefocus_min = -150.0
sim_meta.probeDefocus_max = 150.0
sim_meta.probeDefocus_step = 4.6875
if not pathlib.Path(sim_meta.filenameOutput).exists():
    sim_meta.go()

## Plotting

In [1]:
def rms(img1, img2):
    dif = img1 - img2
    a1 = np.mean(dif**2)
    b1 = np.mean(img2**2)
    return np.sqrt(a1/b1)

In [ ]:
f = h5py.File(fp_20, 'r')
df_dim = np.array(f['4DSTEM_simulation']['data']['supergroups']['vd_CC_series']['sgdim1'])
df_series_20 = np.array(f['4DSTEM_simulation']['data']['supergroups']['vd_CC_series']['supergroup'])
f.close()

f = h5py.File(fp_100, 'r')
df_series_100 = np.array(f['4DSTEM_simulation']['data']['supergroups']['vd_CC_series']['supergroup'])
f.close()

In [ ]:
bfo = 20 #bright field outer angle mrad
abfi = 10 #abf inner angle mrad
adfi = 40 #haadf inner angle
adfo = 90 #haadf outer angle

bf_20 = np.sum(df_series_20[:,:,0:bfo,:],axis=2) #BF 
abf_20 = np.sum(df_series_20[:,:,abfi:bfo,:],axis=2) #ABF
haadf_20 = np.sum(df_series_20[:,:,adfi:adfo,:],axis=2) #HAADF

bf_100 = np.sum(df_series_100[:,:,0:bfo,:],axis=2) #BF 
abf_100 = np.sum(df_series_100[:,:,abfi:bfo,:],axis=2) #ABF
haadf_100 = np.sum(df_series_100[:,:,adfi:adfo,:],axis=2) #HAADF

In [6]:
ws_haadf_20 = []
ws_bf_20 = []
ws_abf_20 = []
ws_haadf_100 = []
ws_bf_100 = []
ws_abf_100 = []

for i in range(df_series_20.shape[-1]):
    ws_haadf_20.append(np.squeeze(haadf_20[:,:,i]))
    ws_bf_20.append(np.squeeze(bf_20[:,:,i]))
    ws_abf_20.append(np.squeeze(abf_20[:,:,i]))
    ws_haadf_100.append(np.squeeze(haadf_100[:,:,i]))
    ws_bf_100.append(np.squeeze(bf_100[:,:,i]))
    ws_abf_100.append(np.squeeze(abf_100[:,:,i]))

In [7]:
sigma_20 = 50
sigma_100 = 10

ws_haadf_cc_20 = Cc(ws_haadf_20, df_dim)
ws_bf_cc_20 = Cc(ws_bf_20, df_dim)
ws_abf_cc_20 = Cc(ws_abf_20, df_dim)

ws_haadf_cc_100 = Cc(ws_haadf_100, df_dim)
ws_bf_cc_100 = Cc(ws_bf_100, df_dim)
ws_abf_cc_100 = Cc(ws_abf_100, df_dim)

haadf_avg_20 = ws_haadf_cc_20.computeImage(0, sigma_20)
bf_avg_20 = ws_bf_cc_20.computeImage(0, sigma_20)
abf_avg_20 = ws_abf_cc_20.computeImage(0, sigma_20)

haadf_avg_100 = ws_haadf_cc_100.computeImage(0, sigma_100)
bf_avg_100 = ws_bf_cc_100.computeImage(0, sigma_100)
abf_avg_100 = ws_abf_cc_100.computeImage(0, sigma_100)

In [7]:
n = [1]
rms1 = [rms(bf_20[:,:,32], bf_avg_20)]
rms2 = [rms(abf_20[:,:,32], abf_avg_20)]
rms3 = [rms(haadf_20[:,:,32], haadf_avg_20)]
rms4 = [rms(bf_100[:,:,32], bf_avg_100)]
rms5 = [rms(abf_100[:,:,32], abf_avg_100)]
rms6 = [rms(haadf_100[:,:,32], haadf_avg_100)]

# Calculate RMSE for different number of defocus planes
for k, j in enumerate([32, 16, 8, 4, 2, 1]):
    
    d = []
    image1 = []
    image2 = []
    image3 = []
    image4 = []
    image5 = []
    image6 = []


    for i in range(2**(k + 1) + 1):
        
        d.append(df_dim[j*i])
        image1.append(ws_bf_20[j*i])
        image2.append(ws_abf_20[j*i])
        image3.append(ws_haadf_20[j*i])
        image4.append(ws_bf_100[j*i])
        image5.append(ws_abf_100[j*i])
        image6.append(ws_haadf_100[j*i])
    
    x1 = Cc(image1, d)
    x2 = Cc(image2, d)
    x3 = Cc(image3, d)
    x4 = Cc(image4, d)
    x5 = Cc(image5, d)
    x6 = Cc(image6, d)
    
    rms1.append(rms(x1.computeImage(0,sigma_20), bf_avg_20))
    rms2.append(rms(x2.computeImage(0,sigma_20), abf_avg_20))
    rms3.append(rms(x3.computeImage(0,sigma_20), haadf_avg_20))
    rms4.append(rms(x4.computeImage(0,sigma_100), bf_avg_100))
    rms5.append(rms(x5.computeImage(0,sigma_100), abf_avg_100))
    rms6.append(rms(x6.computeImage(0,sigma_100), haadf_avg_100))
    n.append(len(x1.plotweights))

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = fig.gca()
n = np.array(n)
ax.loglog(n[:-1], rms1[:-1], color="red", linewidth=1.25, marker="o", label="BF 20kV")
ax.loglog(n[:-1], rms4[:-1], color="red", linestyle="--", linewidth=1.25, marker="o", label="BF 100kV")
ax.loglog(n[:-1], rms3[:-1], color=(0,.6,1), linewidth=1.25, marker="o", label="HAADF 20kV")
ax.loglog(n[:-1], rms6[:-1], color=(0,.6,1), linestyle="--", linewidth=1.25, marker="o", label="HAADF 100kV")

ax.set_ylabel("RMS error (fraction of probe)", fontsize=14)
ax.set_xlabel("Number of additional defocus planes", fontsize=14)
xt = np.array([2**x for x in range(0,6)])
xtl = np.copy(xt)
xtl[0] = 0
xt[1:] += 1
ax.set_xticks(xt)
ax.set_xticklabels(xtl)
ax.grid(True)
plt.setp(ax.spines.values(), linewidth=1.5)
ax.legend(frameon=False, loc="lower left")